In [2]:
import json
import os
from os.path import join
import csv
import random
from lxml import etree
from fuzzywuzzy import fuzz
import re
from collections import defaultdict as dd
from tqdm import tqdm

import utils
import settings

import logging

import os

import numpy as np

import warnings
warnings.filterwarnings('ignore')

import pandas as pd
import ast
import json
from tqdm import tqdm
from lxml import etree
import os
import glob

import networkx as nx
from tqdm import tqdm
import torch_cluster
import torch
from torch_geometric.nn import Node2Vec
from torch_geometric.utils.convert import from_networkx
from random import sample
import numpy as np
import umap.umap_ as umap
import pandas as pd
import joblib  

logger = logging.getLogger(__name__)
logging.basicConfig(level=logging.INFO, format='%(asctime)s %(message)s')  # include timestamp

import pandas as pd

random.seed(1)

dir_path = '../data/PST/'

In [3]:
dblp_df = pd.read_pickle(dir_path + 'OAG_DBLP-Citation-network.pkl')

In [8]:
xml_analysis = pd.read_pickle(dir_path + 'dataset_v2.pkl')

valid_data = pd.read_json(dir_path + 'paper_source_trace_valid_wo_ans.json')
train_data = pd.read_json(dir_path + 'paper_source_trace_train_ans.json')
test_data = pd.read_json(dir_path + 'paper_source_trace_test_wo_ans.json')

### basic feature set

In [9]:
# Convert 'n_citation' column to numeric, coercing errors to NaN
dblp_df['n_citation'] = pd.to_numeric(dblp_df['n_citation'], errors='coerce')

# Calculate the average citation count per venue to estimate venue impact
venue_impact = dblp_df.groupby('venue')['n_citation'].mean().sort_values(ascending=False)

(venue_impact.reset_index()).to_pickle(dir_path + 'venue_impact_v2.pkl')

target_paper_ids = list(set(list(valid_data._id)+
                            list(train_data._id)
                            +list(test_data._id)+
                           list(xml_analysis.loc[xml_analysis['target'] == 1]._id)))

dblp_df['references'] = dblp_df['references'].apply(lambda x: x if isinstance(x, str) else str(x))

# Create a dictionary to hold the count of how many times each paper is cited
citation_counts = dblp_df['references'].explode().value_counts().to_dict()
# Map the citation counts to the original dataframe using the 'id' column
dblp_df['in_citation_count'] = dblp_df['id'].map(citation_counts).fillna(0)

dblp_df['page_start'] = pd.to_numeric(dblp_df['page_start'], errors='coerce')
dblp_df['page_end'] = pd.to_numeric(dblp_df['page_end'], errors='coerce')

# ページ数を計算
dblp_df['page_count'] = dblp_df['page_end'] - dblp_df['page_start'] + 1

# データを確認
dblp_df[['id', 'title', 'page_start', 'page_end', 'page_count']].head()


dblp_df[['id','in_citation_count','page_count']].to_pickle(dir_path + 'dblp_feature_v2.pkl')

df = dblp_df[['id','authors','n_citation']].copy()

import pandas as pd
from tqdm import tqdm  

def expand_authors(row):
    authors = row['authors']
    if isinstance(authors, str):
        authors = eval(authors)
    
    if authors:
        first_author = authors[0]
        last_author = authors[-1]
        authors_extracted = []
        if first_author:
            authors_extracted.append({'id': row['id'], 'author_id': first_author.get('id', ''), 'author_type': 'first_author'})
        if last_author and last_author != first_author:
            authors_extracted.append({'id': row['id'], 'author_id': last_author.get('id', ''), 'author_type': 'last_author'})
        return authors_extracted
    return []

expanded_authors = pd.concat([pd.DataFrame(expand_authors(row)) for index, row in tqdm(df.iterrows(), total=df.shape[0])], ignore_index=True)

print(expanded_authors.head())


df = df.merge(expanded_authors, on = 'id', how = 'left')

temp = df[['author_id','n_citation']].groupby(['author_id']).sum().reset_index()

temp2 = temp.merge(df[['id','author_id']], how = 'left',on = 'author_id')
temp2 = temp2.drop_duplicates()

temp2 = temp2[['id','n_citation']].groupby(['id']).sum().reset_index()

temp2.to_pickle(dir_path + 'author_citation_v2.pkl')

100%|██████████| 6999535/6999535 [50:39<00:00, 2303.17it/s]  


                         id                 author_id   author_type
0  5390877920f70186a0d2cb7c  53f42ed4dabfaedd74d498b4  first_author
1  5390877920f70186a0d2cc08  53f45948dabfaee02ad63b9c  first_author
2  5390877920f70186a0d2cc08  560bdcad45cedb3397402156   last_author
3  5390877920f70186a0d2ce7f  56055d5945cedb33965f78ad  first_author
4  5390877920f70186a0d2ce7f  548a62d3dabfae8a11fb49e2   last_author


### node2vec

In [20]:
dblp_df = pd.read_pickle(dir_path + 'OAG_DBLP-Citation-network.pkl')

In [ ]:
efficient_filtered_network = nx.DiGraph()

for index, row in tqdm(dblp_df.iterrows(), total=dblp_df.shape[0], desc="Building Network"):
    references = row['references']
    if isinstance(references, str):
        references = eval(references)
    
    if references: 
        efficient_filtered_network.add_node(row['id'])
        for ref_id in references:
            efficient_filtered_network.add_edge(row['id'], ref_id)
            if ref_id not in efficient_filtered_network:
                efficient_filtered_network.add_node(ref_id)

print("Number of nodes:", efficient_filtered_network.number_of_nodes())
print("Number of edges:", efficient_filtered_network.number_of_edges())
print("Sample nodes:", list(efficient_filtered_network.nodes())[:5])
print("Sample edges:", list(efficient_filtered_network.edges())[:5])

In [ ]:
torch_graph = from_networkx(efficient_filtered_network)

node_ids = list(efficient_filtered_network.nodes())

t = node_ids

os.environ["CUDA_VISIBLE_DEVICES"] = "0"
os.environ["TOKENIZERS_PARALLELISM"] = "false"


device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = Node2Vec(torch_graph.edge_index, embedding_dim=64, walk_length=10,
                 context_size=5, walks_per_node=5, num_negative_samples=5,
                 p=1, q=1.1, sparse=True)

if torch.cuda.device_count() > 1:
    print(f"{torch.cuda.device_count()} GPUs が利用可能です。DataParallelを使用します。")
    model = torch.nn.DataParallel(model)
    
model.to(device)

loader = model.module.loader(batch_size=128, shuffle=True, num_workers=4) if isinstance(model, torch.nn.DataParallel) else model.loader(batch_size=128, shuffle=True, num_workers=4)
optimizer = torch.optim.SparseAdam(list(model.parameters()), lr=0.01)

def train():
    model.train()
    total_loss = 0
    for pos_rw, neg_rw in loader:
        optimizer.zero_grad()

        loss = model.module.loss(pos_rw.to(device), neg_rw.to(device)) if isinstance(model, torch.nn.DataParallel) else model.loss(pos_rw.to(device), neg_rw.to(device))
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    return total_loss / len(loader)

# Train the model
for epoch in range(7):
    loss = train()
    print(f'Epoch {epoch + 1}, Loss: {loss:.4f}')

# Extract embeddings
with torch.no_grad():
    model.eval()
    embeddings = model()

print(embeddings.shape)  # Checking the shape of the embeddings

In [ ]:

sample_size = 2000000  # サンプルサイズは状況に応じて調整
indices = np.random.choice(embeddings.shape[0], sample_size, replace=False)
sampled_embeddings = embeddings[indices]
sampled_embeddings_cpu = sampled_embeddings.detach().cpu().numpy()  # ここを修正

umap_model = umap.UMAP(n_neighbors=30, n_components=2, min_dist=0.0, metric='euclidean', random_state=42)
umap_model.fit(sampled_embeddings_cpu) 

sampled_embeddings_cpu = embeddings.detach().cpu().numpy()  

sampled_embedding_reduced = umap_model.transform(sampled_embeddings_cpu)

df_full = pd.DataFrame(sampled_embedding_reduced, columns=['UMAP1', 'UMAP2'])

df_full['node_id'] = t
df_full[['UMAP1','UMAP2','node_id']].to_csv(dir_path + 'node2vec_umap_v3.csv.gz', compression='gzip')